In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.20.0


In [ ]:
# Paths and parameters
train_dir = 'train_1'
test_dir = 'test_1'
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 15
# Use TensorFlow SavedModel directory (not .h5).
MODEL_PATH = 'exercise_6_trained_model_improved'
# Also keep an H5 filename for compatibility (renamed to exercise_6_resnet.h5)
H5_PATH = 'exercise_6_resnet.h5'

In [3]:
# Data generators with augmentation for training and simple rescaling for validation/test
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [4]:
# Build the model using ResNet50 as base (transfer learning)
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # freeze base for initial training (fast)

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)

model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 4, 4, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Training with callbacks (EarlyStopping).
# We avoid ModelCheckpoint saving an .h5 file here; the notebook will explicitly
# save the final model in TensorFlow SavedModel format after training.
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)
]

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks
)

Epoch 1/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5052 - loss: 0.7951

119/119 ━━━━━━━━━━━━━━━━━━━━ 267s 2s/step - accuracy: 0.5034 - loss: 0.7791 - val_accuracy: 0.4868 - val_loss: 0.7036
Epoch 2/15
Epoch 2/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5156 - loss: 0.7566

119/119 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.5238 - loss: 0.7481 - val_accuracy: 0.7365 - val_loss: 0.6865
Epoch 3/15
Epoch 3/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5286 - loss: 0.7392

119/119 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.5341 - loss: 0.7312 - val_accuracy: 0.7206 - val_loss: 0.6774
Epoch 4/15
Epoch 4/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5390 - loss: 0.7224

119/119 ━━━━━━━━━━━━━━━━━━━━ 229s 2s/step - accuracy: 0.5430 - loss: 0.7136 - val_accuracy: 0.6487 - val_loss: 0.6674
Epoch 5/15
Epoch 5/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5633 - loss: 0.6976

...

119/119 ━━━━━━━━━━━━━━━━━━━━ 574s 5s/step - accuracy: 0.6032 - loss: 0.6655 - val_accuracy: 0.7513 - val_loss: 0.6406
Epoch 12/15
Epoch 12/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6019 - loss: 0.6633

119/119 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step - accuracy: 0.5993 - loss: 0.6647 - val_accuracy: 0.7672 - val_loss: 0.6370
Epoch 13/15
Epoch 13/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6220 - loss: 0.6553

119/119 ━━━━━━━━━━━━━━━━━━━━ 210s 2s/step - accuracy: 0.6106 - loss: 0.6596 - val_accuracy: 0.7810 - val_loss: 0.6296
Epoch 14/15
Epoch 14/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6075 - loss: 0.6580

119/119 ━━━━━━━━━━━━━━━━━━━━ 354s 3s/step - accuracy: 0.6067 - loss: 0.6586 - val_accuracy: 0.7778 - val_loss: 0.6259
Epoch 15/15
Epoch 15/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6230 - loss: 0.6591

119/119 ━━━━━━━━━━━━━━━━━━━━ 173s 1s/step - accuracy: 0.6175 - loss: 0.6605 - val_accuracy: 0.7312 - val_loss: 0.6198


In [6]:
# Optional: unfreeze some of the top layers for fine-tuning (uncomment to use)
# base_model.trainable = True
# for layer in base_model.layers[:-10]:
#     layer.trainable = False
# model.compile(optimizer=optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
# history_fine = model.fit(train_generator, epochs=5, validation_data=val_generator, callbacks=callbacks)

In [7]:
# Evaluate on test set
if 'test_generator' in globals():
    test_loss, test_acc = model.evaluate(test_generator, verbose=1)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')
else:
    print('Test generator not found; check test_dir path')

37/37 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.7407 - loss: 0.6183
37/37 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.7407 - loss: 0.6183
Test Loss: 0.6183, Test Accuracy: 0.7407
Test Loss: 0.6183, Test Accuracy: 0.7407


In [ ]:
# Inference helpers: return label and confidence
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model_path=H5_PATH):
    # Try loading the H5 file first (fast), otherwise fall back to the in-memory model or SavedModel dir
    if os.path.exists(model_path):
        m = tf.keras.models.load_model(model_path)
    elif os.path.exists(MODEL_PATH):
        m = tf.keras.models.load_model(MODEL_PATH)
    else:
        m = model
    img = image.load_img(img_path, target_size=IMG_SIZE)
    arr = image.img_to_array(img) / 255.0
    arr = np.expand_dims(arr, 0)
    pred = float(m.predict(arr)[0,0])
    label = 'Chihuahua' if pred >= 0.5 else 'Muffin'
    return label, pred

def pretty_predict(img_path, model_path=H5_PATH):
    if not os.path.exists(img_path):
        print('Image not found:', img_path); return
    label, conf = predict_image(img_path, model_path=model_path)
    print(f'{img_path} -> {label} (confidence: {conf:.4f})')

In [9]:
# Predict the run_1 images (if they exist)
pretty_predict('run_1/run_1.jpg')
pretty_predict('run_1/run_2.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
run_1/run_1.jpg -> Chihuahua (confidence: 0.5272)
run_1/run_1.jpg -> Chihuahua (confidence: 0.5272)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
run_1/run_2.jpg -> Chihuahua (confidence: 0.5911)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
run_1/run_2.jpg -> Chihuahua (confidence: 0.5911)


In [ ]:
# Save final model explicitly in TensorFlow SavedModel format (directory) and as H5
model.save(MODEL_PATH, save_format='tf')
# Also save an H5 file with the requested name for compatibility/assignment
model.save(H5_PATH)
print('Saved model as SavedModel directory:', MODEL_PATH)
print('Saved model as H5:', H5_PATH)

Saved model as exercise_6_trained_model_improved.h5
